<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Process-Data" data-toc-modified-id="Process-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Process Data</a></span></li><li><span><a href="#Pre-Process-Data-For-Deep-Learning" data-toc-modified-id="Pre-Process-Data-For-Deep-Learning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pre-Process Data For Deep Learning</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Look-at-one-example-of-processed-issue-bodies" data-toc-modified-id="Look-at-one-example-of-processed-issue-bodies-2.0.0.1"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Look at one example of processed issue bodies</a></span></li><li><span><a href="#Look-at-one-example-of-processed-issue-titles" data-toc-modified-id="Look-at-one-example-of-processed-issue-titles-2.0.0.2"><span class="toc-item-num">2.0.0.2&nbsp;&nbsp;</span>Look at one example of processed issue titles</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Define-Model-Architecture" data-toc-modified-id="Define-Model-Architecture-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define Model Architecture</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-the-data-from-disk-into-variables" data-toc-modified-id="Load-the-data-from-disk-into-variables-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Load the data from disk into variables</a></span></li><li><span><a href="#Define-Model-Architecture" data-toc-modified-id="Define-Model-Architecture-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>Define Model Architecture</a></span></li></ul></li></ul></li><li><span><a href="#Train-Model" data-toc-modified-id="Train-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train Model</a></span></li><li><span><a href="#See-Results-On-Holdout-Set" data-toc-modified-id="See-Results-On-Holdout-Set-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>See Results On Holdout Set</a></span></li><li><span><a href="#Feature-Extraction-Demo" data-toc-modified-id="Feature-Extraction-Demo-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Feature Extraction Demo</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Example-1:-Issues-Installing-Python-Packages" data-toc-modified-id="Example-1:-Issues-Installing-Python-Packages-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Example 1: Issues Installing Python Packages</a></span></li><li><span><a href="#Example-2:--Issues-asking-for-feature-improvements" data-toc-modified-id="Example-2:--Issues-asking-for-feature-improvements-6.0.2"><span class="toc-item-num">6.0.2&nbsp;&nbsp;</span>Example 2:  Issues asking for feature improvements</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
import tensorflow as tf
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError: # If TPU not found
  tpu = None
# Select appropriate distribution strategy
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
else:
  strategy = tf.distribute.get_strategy() # Default strategy that works on CPU and single GPU
  print('Running on CPU instead')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.30.40.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.30.40.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.30.40.226:8470']
Number of accelerators:  8


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'seq2seq-model/'
import sys
sys.path.append(base_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pip install astor
!pip install wget
!pip install nmslib
!pip install pathos
!pip install pandas
!pip install more_itertools
!pip install spacy
!python -m spacy download en
!pip install ktext
!pip install annoy
!pip install fastai
!pip install torch
!pip install torchtext
!pip install torchvision
import tqdm
from tqdm import tqdm_notebook

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=c7b9ceaa026870203c185f4b162d1b182240228709d64b948b11a1db66d2430c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 13.0MB 303kB/s 
     |████████████████████████████████| 296kB 43.2MB/s 
     |████████████████████████████████| 225kB 3.4MB/s 
     |████████████████████████████████| 112kB 11.0MB/s 
     |████████████████████████████████| 184kB 10.4MB/s 
     |████████████████████████████████| 133kB 10.8MB/s 
     |████████████████████████████████| 2.4MB 11.1MB/s 
  Created wheel for pathos: filename=pathos-0.2.6-cp36-none-any.whl size=77673 sha256=8944ae0aaa4ea00e57493a36f8bb7076a95110a8369c60978495a414674a02bd
  Stored in directory: /root/.cache/pip/wheels/3a/e8/c8/04cdd0c4bc6fbce35f642fc004244228916daae74bb0f482da
  Created wheel for ppft: filename=ppft-1.6.6.2-cp36-none-any.whl size=64743

In [ ]:
import pandas as pd
import logging
import glob
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 500)
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [ ]:
from pathlib import Path
INPUT_PATH = Path(base_dir)
INPUT_FILE = Path(INPUT_PATH/'github_issues.csv')
print("INPUT_FILE = ", INPUT_FILE)

INPUT_FILE =  /content/gdrive/My Drive/seq2seq-model/github_issues.csv


# Process Data

Look at filesystem to see files extracted from BigQuery (or Kaggle: https://www.kaggle.com/davidshinn/github-issues/)

In [ ]:
!ls -lah | grep INPUT_FILE

Split data into train and test set and preview data

In [ ]:
#read in data sample 2M rows (for speed of tutorial)
traindf, testdf = train_test_split(pd.read_csv(INPUT_FILE).sample(n=200000), 
                                   test_size=.10)


#print out stats about shape of data
print(f'Train: {traindf.shape[0]:,} rows {traindf.shape[1]:,} columns')
print(f'Test: {testdf.shape[0]:,} rows {testdf.shape[1]:,} columns')

# preview data
traindf.head(3)

Train: 180,000 rows 3 columns
Test: 20,000 rows 3 columns


,issue_url,issue_title,body
3372507,"""https://github.com/jmarceli/array2xml/issues/8""",always use cdata for tag content,"for the sake of consistency, wouldn't be better to always use cdata blocks for the tag content? in this way, the value supplied can always be used as is, and will never require any escaping even if performed automatically . also, this would get rid of the prefix. finally, when a tag has both attributes and a value, it should be possible to get rid of both ! and % prefixes by using the first value with numeric key as the value for the tag, that is, instead of 'photos' => array 'photo' => arra..."
855380,"""https://github.com/MoePlayer/DPlayer/issues/120""","if i want to add svg icon , how should i do ?","if i want to add svg icon , how should i do ? / svg used by dplayer / const svgsource = { 'play': '0 0 16 32', 'm15.552 15.168q0.448 0.32 0.448 0.832 0 0.448-0.448 0.768l-13.696 8.512q-0.768 0.512-1.312 0.192t-0.544-1.28v-16.448q0-0.96 0.544-1.28t1.312 0.192z' }; how create the play svg icon data?"
3157007,"""https://github.com/AndreiGubskii/html_group_8_teamwork_team_5/issues/28""",собрать и протестировать страницу singlework,"<h2> задача</h3> - cобрать и протестировать страницу макета https://drive.google.com/drive/folders/0b_ucljvtcpnlwu1ptfitvwjrd2s . - блоки макета должны корректно отображаться - адаптивная страница - блоки не ломаются при резайзе экрана. - если найдете баг или не соответствие блоков по дизайну макета, обращайтесь в чат нашей команды. <h2>время</h2> по плану: 2 часа по факту:"


**Convert to lists in preparation for modeling**

In [ ]:
train_body_raw = traindf.body.tolist()
train_title_raw = traindf.issue_title.tolist()
#preview output of first element
train_body_raw[0]

"for the sake of consistency, wouldn't be better to always use cdata blocks for the tag content? in this way, the value supplied can always be used as is, and will never require any escaping even if performed automatically . also, this would get rid of the prefix. finally, when a tag has both attributes and a value, it should be possible to get rid of both ! and % prefixes by using the first value with numeric key as the value for the tag, that is, instead of 'photos' => array 'photo' => array 0 => array '@mainphoto' => '1', '%' => '1.png', , 1 => array '@mainphoto' => '0', '%' => '2.png', , 2 => array '@mainphoto' => '0', '%' => '3.png', it should be sufficient to write 'photos' => array 'photo' => array 0 => array '@mainphoto' => '1', '1.png', , 1 => array '@mainphoto' => '0', '2.png', , 2 => array '@mainphoto' => '0', '3.png',"

# Pre-Process Data For Deep Learning

See [this repo](https://github.com/hamelsmu/ktext) for documentation on the ktext package

In [ ]:
%reload_ext autoreload
%autoreload 2
from ktext.preprocess import processor

In [ ]:
%%time
# Clean, tokenize, and apply padding / truncating such that each document length = 70
#  also, retain only the top 8,000 words in the vocabulary and set the remaining words
#  to 1 which will become common index for rare words 
with strategy.scope(): 
  body_pp = processor(keep_n=8000, padding_maxlen=70)
  train_body_vecs = body_pp.fit_transform(train_body_raw)

#### Look at one example of processed issue bodies

In [ ]:
print('\noriginal string:\n', train_body_raw[0], '\n')
print('after pre-processing:\n', train_body_vecs[0], '\n')

NameError: ignored

In [ ]:
# Instantiate a text processor for the titles, with some different parameters
#  append_indicators = True appends the tokens '_start_' and '_end_' to each
#                      document
#  padding = 'post' means that zero padding is appended to the end of the 
#             of the document (as opposed to the default which is 'pre')
title_pp = processor(append_indicators=True, keep_n=4500, 
                     padding_maxlen=12, padding ='post')

# process the title data
train_title_vecs = title_pp.fit_transform(train_title_raw)

NameError: ignored

#### Look at one example of processed issue titles

In [ ]:
print('\noriginal string:\n', train_title_raw[0])
print('after pre-processing:\n', train_title_vecs[0])

In [ ]:
OUTPUT_PATH = Path(base_dir +'./data/seq2seq/')
OUTPUT_PATH.mkdir(exist_ok=True)

Serialize all of this to disk for later use

In [ ]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open(OUTPUT_PATH/'body_pp.dpkl', 'wb') as f:
    dpickle.dump(body_pp, f)

with open(OUTPUT_PATH/'title_pp.dpkl', 'wb') as f:
    dpickle.dump(title_pp, f)

# Save the processed data
np.save(OUTPUT_PATH/'train_title_vecs.npy', train_title_vecs)
np.save(OUTPUT_PATH/'train_body_vecs.npy', train_body_vecs)

# Define Model Architecture

### Load the data from disk into variables

In [ ]:
from seq2seq_utils import load_decoder_inputs, load_encoder_inputs, load_text_processor

In [ ]:
encoder_input_data, doc_length = load_encoder_inputs(OUTPUT_PATH/'train_body_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs(OUTPUT_PATH/'train_title_vecs.npy')

In [ ]:
num_encoder_tokens, body_pp = load_text_processor(OUTPUT_PATH/'body_pp.dpkl')
num_decoder_tokens, title_pp = load_text_processor(OUTPUT_PATH/'title_pp.dpkl')

### Define Model Architecture

In [ ]:
%matplotlib inline
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, BatchNormalization
from tensorflow.keras import optimizers
from seq2seq_utils import viz_model_architecture
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
with strategy.scope(): 
  #arbitrarly set latent dimension for embedding and hidden units
  latent_dim = 300

  ##### Define Model Architecture ######

  ########################
  #### Encoder Model ####
  encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')

  # Word embeding for encoder (ex: Issue Body)
  x = Embedding(num_encoder_tokens, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
  x = BatchNormalization(name='Encoder-Batchnorm-1')(x)

  # Intermediate GRU layer (optional)
  #x = GRU(latent_dim, name='Encoder-Intermediate-GRU', return_sequences=True)(x)
  #x = BatchNormalization(name='Encoder-Batchnorm-2')(x)

  # We do not need the `encoder_output` just the hidden state.
  _, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)

  # Encapsulate the encoder as a separate entity so we can just 
  #  encode without decoding if we want to.
  encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

  seq2seq_encoder_out = encoder_model(encoder_inputs)

  ########################
  #### Decoder Model ####
  decoder_inputs = Input(shape=(None,), name='Decoder-Input')  # for teacher forcing

  # Word Embedding For Decoder (ex: Issue Titles)
  dec_emb = Embedding(num_decoder_tokens, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
  dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

  # Set up the decoder, using `decoder_state_input` as initial state.
  decoder_gru = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')
  decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
  x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

  # Dense layer for prediction
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
  decoder_outputs = decoder_dense(x)

  ########################
  #### Seq2Seq Model ####

  #seq2seq_decoder_out = decoder_model([decoder_inputs, seq2seq_encoder_out])
  
  
  

In [ ]:
with strategy.scope(): 
  seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')
  seq2seq_Model.summary() 
  script_name_base = 'tutorial_seq2seq'
  csv_logger = CSVLogger('{:}.log'.format(script_name_base))
  model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)

  batch_size = 1200
  epochs = 7
  history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

** Examine Model Architecture Summary **

# Train Model

In [ ]:
#save model
seq2seq_Model.save(OUTPUT_PATH/'seq2seq_model_tutorial.h5')

# See Results On Holdout Set

In [ ]:
from seq2seq_utils import Seq2Seq_Inference
seq2seq_inf = Seq2Seq_Inference(encoder_preprocessor=body_pp,
                                 decoder_preprocessor=title_pp,
                                 seq2seq_model=seq2seq_Model)

In [ ]:
# this method displays the predictions on random rows of the holdout set
seq2seq_inf.demo_model_predictions(n=50, issue_df=testdf)



============== Example # 137237 =================

"https://github.com/envisionnw/upland/issues/90"
Issue Body:
 <a href= https://github.com/ncpn ><img src= https://avatars3.githubusercontent.com/u/9699622?v=3 align= left width= 96 height= 96 hspace= 10 ></img></a> issue by ncpn https://github.com/ncpn _friday mar 17, 2017 at 19:31 gmt_
_originally opened as https://github.com/ncpn/upland/issues/90_ ---- check for odd species after completing the plot. compare with list of prior year's species. paper list 

Original Title:
 end of plot issues - identify odd species

****** Machine Generated Title (Prediction) ******:
 closed species plot not working


============== Example # 132413 =================

"https://github.com/open-organization-ambassadors/open-org-it-culture/issues/38"
Issue Body:
 need to include a specific call to the opensource.com writers list during the announcement part of the book series process. 

Original Title:
 update announcement process

****** Machine Genera

 permanent redirect 308 not supported in ie11

****** Machine Generated Title (Prediction) ******:
 redirect to * number * redirect does not work


============== Example # 197532 =================

"https://github.com/ngrx/platform/issues/49"
Issue Body:
 export const selectfeature = createfeatureselector<featurestate> 'feature' ; ~~~~~~~~~~~~~~~ error ts4023: exported variable 'selectfeature' has or is using name 'memoizedselector' from external module .../ngrx/modules/store/src/selector but cannot be named. 

Original Title:
 memoizedselector needs to be exported as well

****** Machine Generated Title (Prediction) ******:
 export ' ' : ' can not be used in ' module


============== Example # 163719 =================

"https://github.com/aspnet/StaticFiles/issues/211"
Issue Body:
 staticfiles/src/microsoft.aspnet.staticfiles/fileextensioncontenttypeprovider.cs is missing the outlook .msg mimetype - currently manually doing the following: var provider = new fileextensioncontenttypepr


****** Machine Generated Title (Prediction) ******:
 new member does not set the team member


============== Example # 64152 =================

"https://github.com/linuxboss182/SoftEng-2017/issues/84"
Issue Body:
 need 3-4 people to present our application to the class on wednesday. applicants must: - not have presented last week - understand how to use the application - be ready to kick ass remember, you have to present at either this wednesday or the next one, so plan accordingly! 

Original Title:
 iteration 2 presentation

****** Machine Generated Title (Prediction) ******:
 add a new class to the application


============== Example # 69032 =================

"https://github.com/kartoza/qgis.org.za/issues/184"
Issue Body:
 i created a form 'contact' and it seems to work but the form labels do not appear on the form so it is a bit useless. please get the labels to appear and merge and release with other improvements asap 

Original Title:
 form labels not appearing

****** Machin


****** Machine Generated Title (Prediction) ******:
 first from flow in uk south


============== Example # 24718 =================

"https://github.com/sensorario/go-tris/issues/34"
Issue Body:
 move 1 simone : 5 move 2 computer : 2 move 3 simone : 9 move 4 computer : 1 move 5 simone : 3 move 6 computer : 6 move 7 simone : 8 move 8 computer : 7 move 9 simone : 4 

Original Title:
 in this case computer loose

****** Machine Generated Title (Prediction) ******:
 move to * number *


============== Example # 2005 =================

"https://github.com/fossasia/susi_firefoxbot/issues/6"
Issue Body:
 actual behaviour only text response from the server is shown expected behaviour support different types of responses like images, links, tables etc. would you like to work on the issue ? yes 

Original Title:
 support for different types of responses from server

****** Machine Generated Title (Prediction) ******:
 support for different types of response


============== Example # 144769 ===

# Feature Extraction Demo

In [ ]:
# Read All 5M data points
all_data_df = pd.read_csv('github_issues.csv')
# Extract the bodies from this dataframe
all_data_bodies = all_data_df['body'].tolist()

In [ ]:
# transform all of the data using the ktext processor
all_data_vectorized = body_pp.transform_parallel(all_data_bodies)

In [ ]:
# save transformed data
with open('all_data_vectorized.dpkl', 'wb') as f:
    dpickle.dump(all_data_vectorized, f)

In [ ]:
%reload_ext autoreload
%autoreload 2
from seq2seq_utils import Seq2Seq_Inference
seq2seq_inf_rec = Seq2Seq_Inference(encoder_preprocessor=body_pp,
                                    decoder_preprocessor=title_pp,
                                    seq2seq_model=seq2seq_Model)
recsys_annoyobj = seq2seq_inf_rec.prepare_recommender(all_data_vectorized, all_data_df)

### Example 1: Issues Installing Python Packages

In [ ]:
seq2seq_inf_rec.demo_model_predictions(n=1, issue_df=testdf, threshold=1)



============== Example # 13563 =================

"https://github.com/bnosac/pattern.nlp/issues/5"
Issue Body:
 thanks for your package, i can't wait to use it. unfortunately i have issues with the installation. prerequisite is 'first install python version 2.5+ not version 3 '. so this package cant be used with version 3.6 64bit that i have installed? i nevertheless tried to install it using pip, conda is not supported? but got an error: 'syntaxerror: missing parentheses in call to 'print''. besides when i try to run the library in r version 3.3.3. 64 bit i got errors with can_find_python_cmd required_modules = pattern.db : 'error in find_python_cmd......' pattern seems to be written in python but must be used in r, why cant it be used in python? i found another python pattern application that apparently does the same in python: https://pypi.python.org/pypi/pattern how is this related? 

Original Title:
 error installation python

****** Machine Generated Title (Prediction) ******:


,issue_url,issue_title,body,dist
286906,"""https://github.com/scikit-hep/root_numpy/issues/337""",root 6.10/02 and root_numpy compatibility,i am trying to pip install root_pandas and one of the dependency is root_numpy however some weird reasons i am unable to install it even though i can import root in python. i am working on python3.6 as i am more comfortable with it. is root_numpy is not yet compatible with the latest root?,0.694671
314005,"""https://github.com/andim/noisyopt/issues/4""",joss review: installing dependencies via pip,"hi, i'm trying to install noisyopt in a clean conda environment running python 3.5. running pip install noisyopt does not install the dependencies numpy, scipy . i see that you do include a requires keyword argument in your setup.py file, does this need to be install_requires ? as in https://packaging.python.org/requirements/ . also, not necessary if you don't want to, but i think it would be good to include a list of dependences somewhere in the readme.",0.698265
48120,"""https://github.com/turi-code/SFrame/issues/389""",python 3.6 compatible,"hi: i tried to install sframe using pip and conda but i can not find anything that will work with python 3.6? has sframe been updated to work with python 3.6 yet? thanks, drew",0.718715


### Example 2:  Issues asking for feature improvements

In [ ]:
seq2seq_inf_rec.demo_model_predictions(n=1, issue_df=testdf, threshold=1)



============== Example # 157322 =================

"https://github.com/Chingu-cohorts/devgaido/issues/89"
Issue Body:
 right now, your profile link is https://devgaido.com/profile. this is fine, but it would be really cool if there was a way to share your profile with other people. on my portfolio, i have social media buttons to freecodecamp, github, ect. without a custom link, i cannot show-off what i have done on devgaido to future employers. 

Original Title:
 feature request: sharable profile.

****** Machine Generated Title (Prediction) ******:
 add a link to your profile

**** Similar Issues (using encoder embedding) ****:



,issue_url,issue_title,body,dist
250423,"""https://github.com/ParabolInc/action/issues/1379""",integrations list view discoverability,"issue - enhancement i was initially confused by the link to my account copy; seeing github in the integrations list made me think it had already been set up . i realize now that i had to allow parabol to post as me. i think that link to my account could use a tooltip explaining what link means, and why you'd want to do so. <img width= 728 alt= screen shot 2017-09-29 at 10 52 05 am src= https://user-images.githubusercontent.com/2146312/31024786-2fd39c46-a50e-11e7-9f2a-6d4a5ed2baeb.png >",0.748828
222304,"""https://github.com/viosey/hexo-theme-material/issues/166""",allow us to use sns-share for github,"i'd love to be able to add a link at the bottom of the page for my github account. however, the sns-share option doesn't currently seem to be able to do this.",0.774398
153327,"""https://github.com/tobykurien/GoogleApps/issues/31""",drive provide download ability,sometimes people share files via g drive. provided a link this app can show some info about the files but doesn't show the download button. i hope that it can be fixed and users would be able to download files with this app.,0.778953


In [ ]:
# incase you need to reset the rec system
# seq2seq_inf_rec.set_recsys_annoyobj(recsys_annoyobj)
# seq2seq_inf_rec.set_recsys_data(all_data_df)

# save object
recsys_annoyobj.save('recsys_annoyobj.pkl')

True